In [10]:
import torch
from torch import Tensor
from torch.utils.data import dataset
from torchtext import datasets
from grok.transformer import Transformer

from torchtext.datasets import PennTreebank
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

from typing import Tuple, Dict
import math

In [11]:
train_iter = datasets.PennTreebank(root="../data", split="train")
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> torch.Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


In [12]:
train_data = data_process(train_iter)
train_data.train = True
train_data.shape

torch.Size([924412])

In [13]:
def data_process(raw_text_iter: dataset.IterableDataset) -> torch.Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


def get_ptb_dataset(train_pct: float, split:str = "train", data_dir:str ="../data"):
        data_iter = datasets.PennTreebank(root=data_dir, split=split)
        tokenizer = get_tokenizer('basic_english')
        
        if split != "train":
            train_iter = datasets.PennTreebank(root=data_dir, split="train")
            vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
        else:
            vocab = build_vocab_from_iterator(map(tokenizer, data_iter), specials=['<unk>'])
        
        vocab.set_default_index(vocab['<unk>'])
        
        data = data_process(data_iter)
        data = data[:int(train_pct*len(data))]
        if split =="train":
            data.train = True
        data.tokenizer = tokenizer
        data.vocab = vocab
        return data


        

In [14]:
def batchify(data: torch.Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).contiguous()
    return data

bptt = 5
def get_batch(source: torch.Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    print(source.shape)
    seq_len = min(bptt, source.size(1) - 1 - i)
    data = source[:,i:i+seq_len]
    target = source[:,i+1:i+1+seq_len]
    return data, target


class PTBIterator(torch.utils.data.IterableDataset):
    """
    An iterator over batches of data in an ArithmeticDataset
    """

    def __init__(
        self,
        dataset: torch.utils.data.Dataset,
        device: torch.device,
        batchsize: float = 2,
        shuffle: bool = True,
    ) -> None:
        """
        :param dataset: the dataset to iterate over
        :param device: the torch device to send batches to
        :param batchsize_hint: * 0 means we use a default batchsize
                               * -1 means the entire dataset
                               * float between 0 and 1 means each batch is
                                 that fraction of the DS
                               * int > 1 means that specific batch size
        :param shuffle: whether or not to randomly shuffle the dataset
        """
        self.dataset = dataset
        self.data = batchify(self.dataset, batchsize)
        self.device = device
        self.reset_iteration(shuffle=shuffle)
        self.batchsize = batchsize

    def reset_iteration(self, shuffle=True):
        self.index = 0
        if shuffle and self.dataset.train:
            self.permutation = torch.randperm(len(self.dataset))
        else:
            self.permutation = torch.arange(len(self.dataset))

    
    def __iter__(self):
        """
        :returns: this iterator
        """
        return self

    def __next__(self) -> Dict[str, Tensor]:
        """
        Returns one batch of data.

        :raises: StopIteration when we're out of data
        :returns: batch tensor of shape (self.batchsize, tokens_per_eq)
        """
        self.index += 1
        return get_batch(self.data, self.index)
        
        
    def __len__(self) -> int:
        """
        :returns: the total number of batches
        """
        return math.ceil(len(self.dataset) / self.batchsize)

    

In [133]:
train_iter = PTBIterator(get_ptb_dataset(0.2), torch.device("cpu"))

In [134]:
len(train_iter)

92441

In [135]:
data,target = next(iter(train_iter))
data, target

torch.Size([2, 92441])


(tensor([[9893, 9894, 9896, 9897, 9898],
         [   0, 1192,    2,    2,    3]]),
 tensor([[9894, 9896, 9897, 9898, 9902],
         [1192,    2,    2,    3,    1]]))

## Test grok internals

In [1]:
from grok.training_ptb import TrainableTransformer
from grok  import training_ptb
import os
import argparse

In [2]:
hparams = argparse.Namespace(random_seed=-1, gpu=0, max_epochs=None, max_steps=1000000, log=False, load_path=None, resume=False, run_path=None, batchsize=512, optimizer='Adam', n_layers=2, n_heads=4, d_model=128, dropout=0.0, weight_noise=0.0, non_linearity='relu', max_context_len=50, dataset='ptb', math_operator='+', operand_length=None, train_data_pct=0.5, warmup_steps=10, anneal_lr_steps=100000, anneal_lr=False, max_lr=0.001, weight_decay=0, weight_decay_kind='to_zero', noise_factor=0, save_activations=False, save_outputs=False, logdir='/home/alpha91/data/codes/grok/nbs/logs', datadir='/home/alpha91/data/codes/grok/nbs/data')




In [3]:
model = TrainableTransformer(hparams).float()

Namespace(random_seed=-1, gpu=0, max_epochs=None, max_steps=1000000, log=False, load_path=None, resume=False, run_path=None, batchsize=512, optimizer='Adam', n_layers=2, n_heads=4, d_model=128, dropout=0.0, weight_noise=0.0, non_linearity='relu', max_context_len=50, dataset='ptb', math_operator='+', operand_length=None, train_data_pct=0.5, warmup_steps=10, anneal_lr_steps=100000, anneal_lr=False, max_lr=0.001, weight_decay=0, weight_decay_kind='to_zero', noise_factor=0, save_activations=False, save_outputs=False, logdir='/home/alpha91/data/codes/grok/nbs/logs', datadir='/home/alpha91/data/codes/grok/nbs/data')


In [4]:
model.val_dataset.shape[0] // 7

10477

In [5]:
train_iter = model.val_dataloader()

In [6]:
batch = next(iter(train_iter))

batch["text"].shape, batch["target"].shape

torch.Size([10477, 7])


(torch.Size([10477, 5]), torch.Size([10477, 5]))

In [7]:
model._step(batch, 0)


Y => torch.Size([10477, 5]) ('N', 'Seq', 'Vocab_size')
Y_hat =>  torch.Size([10477, 5, 9922]) ('N', 'Seq', 'Vocab_size')
Acc y shapes torch.Size([10477, 5]) torch.Size([10477, 5])


(tensor(9.3913, grad_fn=<NllLoss2DBackward0>),
 tensor(0.),
 0.022667382076390182,
 tensor([[  95,  360,    4,  330,   53],
         [ 327, 2465,    4,    1,  664],
         [   0, 2964, 2150,    9,   10],
         ...,
         [   6,    1,  418,   41,  152],
         [  21,   13,    2,   22,    6],
         [   7,   13,    2,   22,    6]]),
 tensor([[[-6.4233e-01,  3.8416e-01,  6.6688e-01,  5.4622e-01,  3.6181e-01],
          [ 3.6725e-01, -1.8352e-01, -9.5994e-01, -4.0968e-01, -1.8899e+00],
          [-7.7870e-01,  2.4395e-02, -8.7754e-01, -8.1040e-01, -8.2959e-01],
          ...,
          [ 4.1714e-01, -3.1220e-01, -4.6067e-01, -4.6338e-01, -6.5327e-01],
          [-6.4522e-01,  5.0497e-01, -4.2418e-02, -4.7216e-04,  5.0809e-01],
          [ 6.4652e-01,  3.1338e-01, -3.2233e-01,  1.6457e-01,  8.9881e-01]],
 
         [[ 7.8595e-01,  4.1528e-01,  1.0589e+00,  3.8252e-01,  2.3192e-02],
          [-8.3560e-01, -8.0072e-01, -7.4234e-01, -6.2866e-01, -8.8343e-01],
          [-6.0286e-0